# Introduction

This notebook shows how to work with [h2o](https://www.h2o.ai/) locally.

The focus is on the [AutoML](https://docs.h2o.ai/h2o-tutorials/latest-stable/h2o-world-2017/automl/index.html) feature.

# Libraries

In [5]:
import h2o
import pandas as pd
from h2o.automl import H2OAutoML
import numpy as np


# Setup 

Start the h2o cluster:

In [3]:
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.13" 2021-10-19; OpenJDK Runtime Environment (build 11.0.13+8-post-Debian-1deb11u1); OpenJDK 64-Bit Server VM (build 11.0.13+8-post-Debian-1deb11u1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.9/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp1r1_ar18
  JVM stdout: /tmp/tmp1r1_ar18/h2o_vscode_started_from_python.out
  JVM stderr: /tmp/tmp1r1_ar18/h2o_vscode_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.7
H2O_cluster_version_age:,12 days
H2O_cluster_name:,H2O_from_python_vscode_ykh4go
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.842 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


# Dataset

Make up some dummy binary classification data:

In [58]:
NUM_OBS = 1000
NUM_FEATS = 5

features = np.random.rand(NUM_OBS, NUM_FEATS)
labels = np.random.randint(0, 2, [NUM_OBS, 1])
data = np.concatenate([labels, features], axis=1)

colnames = ['label'] + [f"f{i + 1}" for i in range(NUM_FEATS)]
df = pd.DataFrame(data, columns=colnames)

df.head(5)


,label,f1,f2,f3,f4,f5
0,1.0,0.855631,0.741465,0.348105,0.524392,0.096047
1,1.0,0.389516,0.681717,0.053762,0.268224,0.448249
2,0.0,0.786234,0.793174,0.976809,0.116824,0.496643
3,1.0,0.757959,0.509505,0.243218,0.989123,0.554604
4,1.0,0.409611,0.727583,0.387726,0.175069,0.192278


Import data to h2o:

In [59]:
df = h2o.H2OFrame(df)
df.describe()


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Rows:1000
Cols:6




,label,f1,f2,f3,f4,f5
type,int,real,real,real,real,real
mins,0.0,0.001803500638448696,0.0006155012010888861,0.0003223416343394181,0.00014364246424325433,0.001784265906123239
mean,0.489,0.5071407777958207,0.49945892792882823,0.4846763584711986,0.5059732735682902,0.5115720670907746
maxs,1.0,0.9998488126740508,0.998708073164131,0.9997704316009248,0.9997788121642146,0.9997973051355116
sigma,0.500129112459102,0.29081096147363183,0.2926879813492884,0.28123580413827326,0.28976620545327775,0.2939256615382199
zeros,511,0,0,0,0,0
missing,0,0,0,0,0,0
0,1.0,0.85563078749519,0.7414649683213306,0.3481048311253141,0.5243916858229456,0.0960470548230189
1,1.0,0.3895157081025083,0.6817165648090524,0.05376185985786519,0.2682240157660345,0.4482491882600752
2,0.0,0.7862337138135466,0.7931738901846558,0.9768089293444372,0.11682364072555818,0.4966426013126915


Convert the `label` column to a categorical:

In [60]:
df['label'] = df['label'].asfactor()

df.describe()


Rows:1000
Cols:6




,label,f1,f2,f3,f4,f5
type,enum,real,real,real,real,real
mins,,0.001803500638448696,0.0006155012010888861,0.0003223416343394181,0.00014364246424325433,0.001784265906123239
mean,,0.5071407777958207,0.49945892792882823,0.4846763584711986,0.5059732735682902,0.5115720670907746
maxs,,0.9998488126740508,0.998708073164131,0.9997704316009248,0.9997788121642146,0.9997973051355116
sigma,,0.29081096147363183,0.2926879813492884,0.28123580413827326,0.28976620545327775,0.2939256615382199
zeros,,0,0,0,0,0
missing,0,0,0,0,0,0
0,1,0.85563078749519,0.7414649683213306,0.3481048311253141,0.5243916858229456,0.0960470548230189
1,1,0.3895157081025083,0.6817165648090524,0.05376185985786519,0.2682240157660345,0.4482491882600752
2,0,0.7862337138135466,0.7931738901846558,0.9768089293444372,0.11682364072555818,0.4966426013126915


In [61]:
np.concatenate([labels, features], axis=1)


array([[1.        , 0.85563079, 0.74146497, 0.34810483, 0.52439169,
        0.09604705],
       [1.        , 0.38951571, 0.68171656, 0.05376186, 0.26822402,
        0.44824919],
       [0.        , 0.78623371, 0.79317389, 0.97680893, 0.11682364,
        0.4966426 ],
       ...,
       [0.        , 0.3066971 , 0.75230755, 0.99139135, 0.9002609 ,
        0.38252038],
       [0.        , 0.94819191, 0.40728247, 0.27730908, 0.31347655,
        0.52849458],
       [1.        , 0.59630252, 0.93433843, 0.18625988, 0.54344329,
        0.3826226 ]])

# Modelling

Specify the predictors and label:

In [62]:
y = 'label'

x = df.columns.remove(y)


Build the models!

In [63]:
aml = H2OAutoML(max_models=100, max_runtime_secs=5 * 60)
aml.train(x=x, y=y, training_frame=df)


AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_grid_1_AutoML_2_20220102_205159_model_2


Status of Neuron Layers: predicting label, 2-class classification, bernoulli distribution, CrossEntropy loss, 402 weights/biases, 8.4 KB, 5,990,000 training samples, mini-batch size 1


,,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
0,,1,5,Input,20.0,,,,,,,,,
1,,2,50,RectifierDropout,30.0,0.0,0.0,0.002455,0.011596,0.0,0.030077,0.846153,-1.786186,2.700651
2,,3,2,Softmax,,0.0,0.0,0.00096,0.000596,0.0,0.062318,0.874329,0.252798,0.022578




ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.21966928021403823
RMSE: 0.4686888949122202
LogLoss: 0.6286767239303895
Mean Per-Class Error: 0.3506597193041432
AUC: 0.7181836008628175
AUCPR: 0.7019140028764714
Gini: 0.43636720172563503

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4456809902654736: 


,,0,1,Error,Rate
0,0,231.0,280.0,0.5479,(280.0/511.0)
1,1,75.0,414.0,0.1534,(75.0/489.0)
2,Total,306.0,694.0,0.355,(355.0/1000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.544000,0.056281,0.540000,0.460000,0.605000,0.585000,0.530000
1,auc,0.560754,0.038668,0.514200,0.563909,0.615962,0.573370,0.536331
2,err,0.456000,0.056281,0.460000,0.540000,0.395000,0.415000,0.470000
3,err_count,91.200000,11.256109,92.000000,108.000000,79.000000,83.000000,94.000000
4,f0point5,0.571355,0.033822,0.575581,0.515695,0.606980,0.573529,0.584989
5,f1,0.665576,0.024866,0.682759,0.630137,0.669456,0.652720,0.692810
6,f2,0.800350,0.046809,0.838983,0.809859,0.746269,0.757282,0.849359
7,lift_top_group,1.214161,0.856629,1.000000,2.173913,1.010101,0.000000,1.886792
8,logloss,0.700489,0.014099,0.700609,0.698032,0.683438,0.722641,0.697727
9,max_per_class_error,0.828590,0.197725,0.910000,1.000000,0.594059,0.638889,1.000000



Scoring History: 


,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_r2,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2022-01-02 20:54:02,0.000 sec,None,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2022-01-02 20:54:02,59.769 sec,303030 obs/sec,10.0,1,10000.0,0.507847,0.710425,-0.032136,0.515337,0.498177,1.022495,0.509
2,,2022-01-02 20:54:07,1 min 4.780 sec,575009 obs/sec,2890.0,289,2890000.0,0.482298,0.656516,0.069104,0.684457,0.659692,1.840491,0.343
3,,2022-01-02 20:54:11,1 min 8.881 sec,657591 obs/sec,5990.0,599,5990000.0,0.468689,0.628677,0.120897,0.718184,0.701914,2.044990,0.355



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,f2,1.000000,1.000000,0.269324
1,f3,0.893064,0.893064,0.240524
2,f5,0.711105,0.711105,0.191518
3,f1,0.616742,0.616742,0.166104
4,f4,0.492087,0.492087,0.132531


Check out the leaderboard:

In [64]:
lb = aml.leaderboard
lb.head()

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
DeepLearning_grid_1_AutoML_2_20220102_205159_model_2,0.558426,0.700489,0.529579,0.5,0.502764,0.252772
DeepLearning_grid_1_AutoML_2_20220102_205159_model_1,0.543117,0.701055,0.5212,0.493415,0.503472,0.253484
DeepLearning_grid_1_AutoML_2_20220102_205159_model_3,0.539333,0.798509,0.519335,0.499022,0.532483,0.283538
StackedEnsemble_BestOfFamily_5_AutoML_2_20220102_205159,0.534064,0.757092,0.513098,0.492216,0.524569,0.275173
DeepLearning_grid_1_AutoML_2_20220102_205159_model_4,0.531677,0.739715,0.520635,0.499022,0.516025,0.266282
DeepLearning_grid_1_AutoML_2_20220102_205159_model_7,0.530665,0.705357,0.514051,0.499022,0.505391,0.25542
DRF_1_AutoML_2_20220102_205159,0.530449,0.731999,0.498438,0.498087,0.515723,0.26597
XRT_1_AutoML_2_20220102_205159,0.530047,0.734146,0.500084,0.5,0.515845,0.266096
StackedEnsemble_AllModels_3_AutoML_2_20220102_205159,0.523113,0.695336,0.514436,0.5,0.501024,0.251025
GBM_grid_1_AutoML_2_20220102_205159_model_8,0.520696,0.716121,0.505029,0.499022,0.510304,0.26041


How many models were built?

In [66]:
f"The leaderboard has {lb.nrows} models"

'The leaderboard has 71 models'

# Study A Model

Pick a model to study:

In [68]:
model_id = "StackedEnsemble_AllModels_3_AutoML_2_20220102_205159"
model = h2o.get_model(model_id)
model

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_3_AutoML_2_20220102_205159

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.21638912874070512
RMSE: 0.4651764490391846
LogLoss: 0.6232077313859791
Null degrees of freedom: 999
Residual degrees of freedom: 992
Null deviance: 1385.8103220696778
Residual deviance: 1246.4154627719581
AIC: 1262.4154627719581
AUC: 0.7772181735960204
AUCPR: 0.7590746889858322
Gini: 0.5544363471920408

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.44333934960073684: 


,,0,1,Error,Rate
0,0,263.0,248.0,0.4853,(248.0/511.0)
1,1,67.0,422.0,0.137,(67.0/489.0)
2,Total,330.0,670.0,0.315,(315.0/1000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.443339,0.728214,244.0
1,max f2,0.378040,0.849156,323.0
2,max f0point5,0.471718,0.714878,198.0
3,max accuracy,0.469106,0.722000,203.0
4,max precision,0.833755,1.000000,0.0
5,max recall,0.305961,1.000000,366.0
6,max specificity,0.833755,1.000000,0.0
7,max absolute_mcc,0.468626,0.444248,204.0
8,max min_per_class_accuracy,0.469632,0.720157,202.0
9,max mean_per_class_accuracy,0.468626,0.722174,204.0



Gains/Lift Table: Avg response rate: 48.90 %, avg score: 47.42 %


...

,,0,1,Error,Rate
0,0,0.0,511.0,1.0,(511.0/511.0)
1,1,0.0,489.0,0.0,(0.0/489.0)
2,Total,0.0,1000.0,0.511,(511.0/1000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.285451,0.656817,399.0
1,max f2,0.285451,0.827131,399.0
2,max f0point5,0.285451,0.544665,399.0
3,max accuracy,0.490701,0.548000,195.0
4,max precision,0.683178,1.000000,0.0
5,max recall,0.285451,1.000000,399.0
6,max specificity,0.683178,1.000000,0.0
7,max absolute_mcc,0.490701,0.094841,195.0
8,max min_per_class_accuracy,0.487416,0.531697,203.0
9,max mean_per_class_accuracy,0.490701,0.547341,195.0



Gains/Lift Table: Avg response rate: 48.90 %, avg score: 48.96 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.01,0.618363,1.431493,1.431493,0.70,0.638074,0.700000,0.638074,0.014315,0.014315,43.149284,43.149284,0.008444
1,2,0.02,0.601547,0.613497,1.022495,0.30,0.611166,0.500000,0.624620,0.006135,0.020450,-38.650307,2.249489,0.000880
2,3,0.03,0.586862,1.226994,1.090661,0.60,0.592168,0.533333,0.613803,0.012270,0.032720,22.699387,9.066121,0.005323
3,4,0.04,0.581787,0.613497,0.971370,0.30,0.584284,0.475000,0.606423,0.006135,0.038855,-38.650307,-2.862986,-0.002241
4,5,0.05,0.573252,1.022495,0.981595,0.50,0.577424,0.480000,0.600623,0.010225,0.049080,2.249489,-1.840491,-0.001801
5,6,0.10,0.550440,1.308793,1.145194,0.64,0.559829,0.560000,0.580226,0.065440,0.114519,30.879346,14.519427,0.028414
6,7,0.15,0.535663,0.940695,1.077028,0.46,0.543632,0.526667,0.568028,0.047035,0.161554,-5.930470,7.702795,0.022611
7,8,0.20,0.526757,1.186094,1.104294,0.58,0.531517,0.540000,0.558901,0.059305,0.220859,18.609407,10.429448,0.040820
8,9,0.30,0.510993,1.042945,1.083845,0.51,0.518339,0.530000,0.545380,0.104294,0.325153,4.294479,8.384458,0.049224
9,10,0.40,0.497754,1.145194,1.099182,0.56,0.504410,0.537500,0.535137,0.114519,0.439673,14.519427,9.918200,0.077638


Details about the metalearner:

In [77]:
meta_model = model.metalearner()

In [78]:
meta_model.coef_norm()

{'Intercept': -0.044282628356300924,
 'DeepLearning_grid_1_AutoML_2_20220102_205159_model_2': 0.05508409290946509,
 'DeepLearning_grid_1_AutoML_2_20220102_205159_model_1': 0.0799091525454377,
 'DeepLearning_grid_1_AutoML_2_20220102_205159_model_3': 0.07041491623100568,
 'DeepLearning_grid_1_AutoML_2_20220102_205159_model_4': 0.006914153724367162,
 'DeepLearning_grid_1_AutoML_2_20220102_205159_model_7': 0.0,
 'DRF_1_AutoML_2_20220102_205159': 0.015803889767325916,
 'XRT_1_AutoML_2_20220102_205159': 0.0,
 'GBM_grid_1_AutoML_2_20220102_205159_model_8': 0.002533394304421921,
 'XGBoost_grid_1_AutoML_2_20220102_205159_model_12': 0.0,
 'GLM_1_AutoML_2_20220102_205159': 0.05293347189806759,
 'DeepLearning_grid_1_AutoML_2_20220102_205159_model_9': 0.0,
 'XGBoost_2_AutoML_2_20220102_205159': 0.0,
 'GBM_4_AutoML_2_20220102_205159': 0.0,
 'XGBoost_grid_1_AutoML_2_20220102_205159_model_7': 0.0,
 'XGBoost_grid_1_AutoML_2_20220102_205159_model_3': 0.0,
 'GBM_grid_1_AutoML_2_20220102_205159_model_9': 

# Save Model

Save best model for prediction later:

In [82]:
m = aml.leader
SAVE_PATH = "/tmp/best_model"
h2o.save_model(m, path =SAVE_PATH)

'/tmp/best_model/DeepLearning_grid_1_AutoML_2_20220102_205159_model_2'